<a href="https://colab.research.google.com/github/siyubao/data/blob/master/Data_scraping_for_broadway_%E2%80%93_Gross_and_Silver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Broadway Data
## By Yaakov Bressler

Functions herein provide a means of scraping **"Gross"** and **"Silver"** Broadway data.


---


**Broadway Gross Data Contains:**
*   Show Name
*   Date (describing Week)
*   Week Number
*   Gross ($)
*   Gross Difference (From Week Prior)

*   Gross Potential italicized text ($)
*   Gross  Potential (%)
*   Average Ticket Price
*   Top Ticket Price
*   Seats Sold
*   Seats Available
*   Number of Performances
*   Capacity (%)
*   Capacity Difference (From Week Prior)


**Broadway Silver Data Contains: **
*   Show Name
*   Show Type
*   Theatre Name
*   Theatre Address
*   Production Type
*   Run Type
*   Market
*   Intermission
*   Run Time
*   Date Previews
*   Date Opening
*   Date Closing


### Create a function that grabs links from a page, using a tag to identify value of link

In [0]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import re

def getLinks_tagged_fast(url, tag):
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    links = []
    # set the opening of each link to be...
    tag = tag
    for link in soup.findAll('a', attrs={'href': re.compile(tag)}):
        links.append(link.get('href'))
    return links

# Scrape Gross Data

Create function that gets pages to loop in get data function

In [0]:
import string
import time
import re
import datetime
import pandas as pd
import numpy as np
import urllib
import requests
from bs4 import BeautifulSoup
import datetime

def get_gross_pages(url_base):
    page_base = url_base
    abc = list(string.ascii_lowercase)
    abc.append('1')
    list_loop_az =[]
    for a in abc:
        list_loop_az.append(page_base+a)
    
    #Congrats, you have list_loop_az, pages a - z where all the shows are contained
    
    # Now, write a code that gets all the links:
    def getLinks_tagged(url, tag):
        r = requests.get(url)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        links = []
        # set the opening of each link to be...
        tag = tag
        for link in soup.findAll('a', attrs={'href': re.compile(tag)}):
            links.append(link.get('href'))
        return links

    # Now use the function:
    show_links_nested = []
    tag = 'https://www.broadwayworld.com/grosses/'
    for page in list_loop_az:
        show_links_nested.append(getLinks_tagged(page, tag))
    show_links = sum(show_links_nested, [])
    
    #Congrats, you have show_links, pages where all the shows are contained
    
    # Now, write a code that gets all the content:
    def get_gross(urls):
        # column names
        dfs = []
        cols = ['Week','Week_Numer', 'Gross', 'Gross_Diff', 'Pot_Gross',
                'Potential_Gross_Percent','Average_Paid_Ticket', 'Top_Ticket','Seats_Sold',
                'Total_Seats', 'Performances','Capacity', 'Capacity_Diff']
        for url in urls:
            df = pd.read_html(url)[0]
            df = df.drop([0,1,2,3])
            df = df.dropna(how='all')
            df = df.drop(df.index[0])
            df = df.dropna(thresh = .01 * len(df), axis=1)
            df.columns = cols
            df['Show_Name'] = re.split('^https://www.broadwayworld.com/grosses/*', url)[1].replace("-", " ").title()
            df = df[:-3]
            dfs.append(df)
        finaldf = pd.concat(dfs) 
        return finaldf
    
    broadway = get_gross(show_links)
    
    
    # Save the df
    today = datetime.date.today().strftime('%m-%d-%Y') 
    output_file = 'broadway_grosses {}.csv'.format(today)
    broadway.to_csv(output_file)
    
    return broadway

In [0]:
broadway = get_gross_pages('https://www.broadwayworld.com/grossesbyshow.cfm?letter=')

# Get Show Links through Theatre URLS
(Doesn't provide full list of shows, so don't use...)

This link is your starting point:

https://www.broadwayworld.com/industry-bww-theatres.cfm


In [0]:
import time
import re
import pandas as pd
import numpy as np
import urllib
import requests
from bs4 import BeautifulSoup
import requests
import urllib.request
from inscriptis import get_text

def get_show_link_lists(theatre_url):
#################################################
    def getLinks_tagged_fast(url, tag):
        r = requests.get(url)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        links = []
         # set the opening of each link to be...
        tag = tag
        for link in soup.findAll('a', attrs={'href': re.compile(tag)}):
            links.append(link.get('href'))
        return links
        
#################################################

    url_foo = theatre_url

    tag_theatre = 'https://www.broadwayworld.com/shows/theatre'
    tag_show = '/shows/'
    theatres = getLinks_tagged_fast(url_foo, tag_theatre)
    show_links_nested = []
    for theatre in theatres:
        show_links_nested.append(getLinks_tagged_fast(theatre, tag_show))
    
    show_links = sum(show_links_nested, [])
    show_links = [link for link in show_links if not link.startswith('/shows/shows')]
    page_base = 'https://www.broadwayworld.com'
    show_list_loop =[]
    for show in show_links:
        show_list_loop.append(page_base+show)
    show_links_production = sum(show_links_nested, [])
    show_links_production = [link for link in show_links if not link.startswith('/shows/shows')]

    show_links_nums_pre = []
    show_links_nums = []

    for link in show_links_production:
        show_links_nums_pre.append(re.findall('\d+\.', link))
    show_links_nums_pre = sum(show_links_nums_pre, [])

    for link in show_links_nums_pre:
        show_links_nums.append(re.findall('\d+', link))
    show_links_nums = sum(show_links_nums, [])

    prod_info_base = 'https://www.broadwayworld.com/shows/backstage.php?showid='
    cast_base = 'https://www.broadwayworld.com/shows/cast.php?showid='

    prod_info_show_loop =[]
    for show in show_links_nums:
        prod_info_show_loop.append(prod_info_base+show)

    cast_show_loop =[]
    for show in show_links_nums:
        cast_show_loop.append(cast_base+show)
    
    return[prod_info_show_loop,cast_show_loop]

In [0]:
# will give you a list of two lists....
# First list is for silver function
# Second list is for bronze function

url = 'https://www.broadwayworld.com/industry-bww-theatres.cfm'
show_links = get_show_link_lists(url)
print('The list \'show_links\' contains a total of %d shows' %len(show_links[0]))

# Get links through Years URLS
** This works, so indeed use.**

This link is your starting point:

https://www.broadwayworld.com/browseshows.cfm?showtype=BR

In [0]:
import time
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import requests
import urllib.request
from inscriptis import get_text

def get_show_links_year(year_url):
    
#################################################
    def getLinks_tagged_fast(url2, tag):
        r = requests.get(url2)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        links = []
         # set the opening of each link to be...
        tag = tag
        for link in soup.findAll('a', attrs={'href': re.compile(tag)}):
            links.append(link.get('href'))
        return links
        
#################################################

    url = year_url
    tag_year = 'browseshows.cfm?'
    years = getLinks_tagged_fast(url, tag_year)[1:]
    page_base = 'https://www.broadwayworld.com/'
    years_loop =[]
    for year in years:
        years_loop.append(page_base+year)

    # Now you have all the years..
    tag_show = 'https://www.broadwayworld.com/shows/backstage.php?'
    show_links_nested = []
    for year in years_loop:
        show_links_nested.append(getLinks_tagged_fast(year,tag_show))
    show_links = sum(show_links_nested, [])
    
    cast_links =[]
    #Get all the cast info:
    for link in show_links:
        cast_links.append(link.replace('backstage', 'cast'))
        
    # Now you have all the productions...
    creative_links =[]
    #Get all the cast info:
    for link in show_links:
        creative_links.append(link.replace('backstage', 'creative'))
        

    return[show_links,cast_links,creative_links]

In [0]:
# will give you a list of two lists....
# First list is for silver function (show production information)
# Second list is for bronze function (function not yet devised, will be for Awards and such)

year_url = 'https://www.broadwayworld.com/browseshows.cfm?showtype=BR'
show_links_nested = get_show_links_year(year_url)
print('The list \'show_links_nested\' contains a total of %d shows' %len(show_links_nested[0]))

The list 'show_links_nested' contains a total of 13568 shows


In [0]:
import pandas as pd
show_links_df_pre = pd.DataFrame(show_links_nested)
show_links_df = show_links_df_pre.transpose()
show_links_df.columns=['show_links','cast_links','creative_links']
show_links_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13568 entries, 0 to 13567
Data columns (total 3 columns):
show_links        13568 non-null object
cast_links        13568 non-null object
creative_links    13568 non-null object
dtypes: object(3)
memory usage: 318.1+ KB


In [0]:
import datetime
today = datetime.date.today().strftime('%m-%d-%Y') 
show_links_df.to_csv('show_links_df {}.csv'.format(today))

----

### Import your spreadsheet with show links:

*Better to download the list of shows then upload and continue working, if your kernel freezes and you have to start again, this will save lots of time.*

----


In [0]:
import os
import pandas as pd
path = os.getcwd()
csvfile = '/show_links_df 09-06-2018.csv'
file = (path+csvfile)

In [0]:
show_links_df = pd.read_csv(file)

In [0]:
show_links_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13568 entries, 0 to 13567
Data columns (total 4 columns):
Unnamed: 0        13568 non-null int64
show_links        13568 non-null object
cast_links        13568 non-null object
creative_links    13568 non-null object
dtypes: int64(1), object(3)
memory usage: 424.1+ KB


In [0]:
show_links = show_links_df['show_links'].tolist()
cast_links = show_links_df['cast_links'].tolist()
creative_links = show_links_df['creative_links'].tolist()
show_links_nested = [show_links,cast_links,creative_links]

---

url = 'https://www.broadwayworld.com/browseshows.cfm?showtype=BR'

Applying **show_links_nested** function on the above url results in a list of lists of shows:
    
    show_links_nested
    
List 0 is show production info, list 1 is cast info

---

# Scrape Silver data

### Pre functions that will be built into the main function

---

In [0]:
#Get Visible text
from bs4 import BeautifulSoup
from bs4.element import Comment
import re
import numpy as np
import pandas as pd

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

In [0]:
# YB: This function is a bit sloppy and could be cleaned, but it works.

def text_to_nums_str(input_text_1):
    output_words = []
    num_dic={'One':'1','Two':'2','Three':'3','Four':'4','Five':'5',
             'Six':'6','Seven':'7','Eight':'8','Nine':'9',
             'one':'1', 'two':'2','three':'3','four':'4', 'no':'0', 'No':'0','None':'0','none':'0'}
   
    if pd.notnull(input_text_1) is True:
        for word in input_text_1.split():
            output_word = "".join(str(x) for x in(re.sub(r'\b\w+\b',lambda m: num_dic.get(m.group(), m.group()) , word)))
            output_words.append(output_word)
        output_words = " ".join(str(x) for x in output_words)
    else:
        output_words = np.NaN
    return output_words

def show_runs_text_to_nums_str(input_text_1):
    output_words = []
    num_dic={'One':'1','Two':'2','Three':'3','Four':'4','Five':'5', 'Six':'6',
             'Seven':'7','Eight':'8','Nine':'9','one':'1', 'two':'2','three':'3',
             'four':'4', 'no':'0', 'No':'0','None':'0','none':'0'}
    #num_dic_pair={'hours, with':'00','hours with':'00','hours\nwith':'00','hours\n with':'00'} 
   
    if pd.notnull(input_text_1) is True:
        for word in input_text_1.split():
            output_word = "".join(str(x) for x in(re.sub(r'\b\w+\b',lambda m: num_dic.get(m.group(), m.group()) , word)))
            #output_word = "".join(str(x) for x in(re.sub(r'\b\w+\s\w+\b',lambda m: num_dic_pair.get(m.group(), m.group()) , output_word_pre)))
            output_words.append(output_word)
        output_words = " ".join(str(x) for x in output_words)
    else:
        output_words = np.NaN
    return output_words
    
# An alternative in lambda:
#output_text = " ".join(str(x) for x in([re.sub(r'\b\w+\b',lambda m: num_dic.get(m.group(), m.group()) , s) for s in text.split()]))

def get_minutes_no_intermission(input_text_2):
    z = show_runs_text_to_nums_str(input_text_2)
    if pd.notnull(z) is True:
        run_time = re.findall('\d', z)
        if len(run_time) >= 1:
            if len(run_time) >=2:
                if len(run_time) >= 3:
                    minutes = int(run_time[1]+run_time[2])
                    hour = (int(run_time[0])*60)
                else:
                    minutes = int(run_time[0]+run_time[1])
                    hour = 0
            else:
                minutes = int(run_time[0])
                hour = 0
        else:
            minutes = 0
            hour = 0 
        run_time_pre = hour+minutes
       
    #the code below doesn't do what I want it to do.
        if run_time_pre >0:
            if run_time_pre ==4:
                run_time = 240
            if run_time_pre ==3:
                run_time = 180
            if run_time_pre ==2:
                run_time = 120
            if run_time_pre ==1:
                run_time = 60
            else:    
                run_time = run_time_pre
        else:
            run_time = np.NaN
    else:
        run_time = np.NaN    
    return run_time



def get_minutes_yes_intermission(input_text_2):
    z = show_runs_text_to_nums_str(input_text_2)
    if pd.notnull(z) is True:
        run_time = re.findall('\d', z)
        if len(run_time) >= 2:
            if len(run_time) >=3:
                if len(run_time) >= 4:
                    minutes = int(run_time[1]+run_time[2])
                    hour = (int(run_time[0])*60)
                else:
                    minutes = int(run_time[0]+run_time[1])
                    hour = 0
            else:
                minutes = int(run_time[0])
                hour = 0
        else:
            minutes = 0
            hour = 0 
        run_time_pre = hour+minutes
        #the code below doesn't do what I want it to do.
        if run_time_pre >0:
            if run_time_pre ==4.0:
                run_time == 240
            if run_time_pre ==3.0:
                run_time = 180
            if run_time_pre ==2.0:
                run_time = 120
            if run_time_pre ==1.0:
                run_time = 60
            else:    
                run_time = run_time_pre
        else:
            run_time = np.NaN
    else:
        run_time = np.NaN   
    
    return run_time

In [0]:
# If you want a progress bar in the silver function
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

In [0]:
# Test the progress bar

from time import sleep
import sys

items = list(range(0, len(show_links_nested[0])))
l = len(items)

# Initial call to print 0% progress
printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 70)
for i, item in enumerate(items):
    sys.stdout.write("-") # prints a dash for each iteration of loop
    sys.stdout.flush() # ensures bar is displayed incrementally
    # Do stuff...
    sleep(0.00001)
    # Update Progress Bar
    printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)

Progress: |██████████████████████████████████████████████████| 100.0% Complete---| 0.0% Complete


### Actual function

---

In [0]:
# Utilize function to get silver data from websites using text parsing...
import string
import time
import re
import datetime
import pandas as pd
import numpy as np
import requests as req
import urllib.request
from bs4 import BeautifulSoup
import datetime   
from termcolor import colored
import sys
from termcolor import colored, cprint

def get_silver_data(silver_urls):
    # for progress bar
    l = len(silver_urls)
    dfs_silver_list = []
    start = time.time()
    # for enumerating progress + errors
    n=0
    n_i = n
    lost_rows = 0
    for each_silver_url in silver_urls:
        print('\n')
        
        # Next three lines print progress bar. Omit if you don't want.
        sys.stdout.write("-") # prints a dash for each iteration of loop
        sys.stdout.flush() # ensures bar is displayed incrementally
        printProgressBar(n + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
        
        # Prints which line is being downloaded, helpful if there is an error.
        print('\n\n%d line is being downloaded' %n)
        
        # ML logic:
        try:
            html = urllib.request.urlopen(each_silver_url).read()
            text = text_from_html(html)
            text2 = re.sub(' +',' ',text)
            if 'Update BWW\'s Database ' in text2:
                text3 = text.split('Update BWW\'s Database ')[1]
            if 'SHOW TYPE:' in text3:
                text_use = text3.split('SHOW TYPE:')[0]
                Show_Type_pre = text3.split('SHOW TYPE:')[1].split()[:2]
            else:
                text_use = text3
                Show_Type_pre = text3
            
            # who knew python accepts emojis as text? Easier for the function...
            if '👥' in text_use:
                Show_Name = text_use.split(' 👥')[0].strip()
            else:
                Show_Name = np.NAN
            gowords = ['Play', 'Musical', 'Special', 'Revue', 'Operetta']
            if any(word in Show_Type_pre for word in gowords):
                    Show_Type = ' '.join(filter(lambda x: x in gowords, Show_Type_pre))
            else: Show_Type = np.NAN
            if 'Theatres' in text_use:
                Theatre = text_use.split('Theatres: ')[1].split(' (')[0]
            else:
                Theatre = np.NAN
            
            # What types of productions are valid?
            gowords_prod_type = ['Original','original','Production','production','Revival',
                                 'revival','Premiere','Revised']
            if 'Production Type:' in text_use:
                Prod_Type_pre = text_use.split('Production Type:')[1].split()[:4]
                if any(word in Prod_Type_pre for word in gowords_prod_type):
                    Prod_Type = ' '.join(filter(lambda x: x in gowords_prod_type, Prod_Type_pre))
            else:
                Prod_Type = np.NAN    
            if 'Run Type: ' in text_use:
                Run_Type = text_use.split('Run Type: ')[1].split(' Market:')[0]
            else: Run_Type = np.NAN
            if 'Market: ' in text_use:
                Market_pre = text_use.split('Market: ')[1][:15]
                # there may be additional go words to add...
                gowords = ['West','End', 'Broadway','Off','Broadway', 'Off-Broadway', 'Off-','Regional']
                Market = ' '.join(filter(lambda x: x in gowords, Market_pre.split()))
            else: Market = np.NAN
            print('...')

    ################ Get Theatre Address
            try:
                if ' Seating Chart Schedule' in text_use:
                    Theatre_Address_pre = text_use.split('Theatre Information ')[1]
                    if ' Seating Chart' in Theatre_Address_pre:
                        Theatre_Address = Theatre_Address_pre.split(' Seating Chart')[0]
                else:
                    if ' Get Show News' in text_use:
                        Theatre_Address_pre = text_use.split('Theatre Information ')[1]
                        if ' Get Show News' in Theatre_Address_pre:
                            Theatre_Address = Theatre_Address_pre.split(' Get Show News')[0]    
                    else: Theatre_Address = Theatre_Address = text_use.split('Theatre Information ')[1][:75]
            except:
                Theatre_Address = np.NaN
    
    ################ Get Run Time        
            if 'Running Time: ' in text_use:
                Run_Time_pre = text_use.split('Running Time: ')[1].split(':')[0]
                stopwords = ['Mondays', 'Tuesdays', 'Wednesdays', 'Thursdays', 'Fridays', 'Saturdays', 'Sundays']
                Run_Time_String = ' '.join(filter(lambda x: x not in stopwords, Run_Time_pre.split()))
                if 'intermission' in Run_Time_String:
                    Run_Time_String = Run_Time_String.split('intermission')[0]
            else:
                Run_Time_String = np.NAN
            Run_Time_Minutes = get_minutes_yes_intermission(Run_Time_String)
            if Run_Time_Minutes ==3.0:
                Run_Time_Minutes = 180
            if Run_Time_Minutes ==2.0:
                Run_Time_Minutes = 120
            if Run_Time_Minutes ==1.0:
                Run_Time_Minutes = 60
            
    ################ Get Intermission
            if 'Intermissions: ' in text_use:
                Intermission = text_use.split('Intermissions: ')[1][:1]
                if 'O' in Intermission:
                    Intermission = np.NAN 
            else:
                Intermission = np.NAN 
                
    ################ Get Opening Date 
            regex = r'(((January)|(February)|(March)|(April)|(May)|(June)|(July)|(August)|(September)|(October)|(November)|( November)|(December))((\d\d,\s)|(\d\d,)|(\s\d\d,\s)|(\s\d\d,))((\d\d\d\d)|(\d\d\/\d\d\/\d\d\d\d)))'
                    
            try:
                if 'Previews: ' in text_use:
                    pre1 = text_use.split('Previews: ')[1]
                    if 'unknown' not in pre1[:10]:
                        Date_Previews = re.match(regex,pre1).group(0)
                    else:
                        Date_Previews = np.NAN
            except:
                Date_Previews = np.NAN
            if 'Opening: ' in text_use:
                pre2 = text_use.split('Opening: ')[1]
                if 'unknown' not in pre2.split()[:2]:
                    Date_Open = re.match(regex,pre2).group(0)
            else:
                Date_Open = np.NAN
    
            if 'Closing: ' in text_use:
                pre3 = text_use.split('Closing: ')[1]
                if 'unknown' not in pre3.split()[:2]:
                    Date_Close = re.match(regex,pre3).group(0)
            else:
                Date_Close = np.NAN
        #    #months = ['January','February','March','April','May','June','July','August','September','October','November','December']   
        #if any(match.group() for match in re.finditer(r"\bmonths[\w]*", Date)):

            # compile as a dictionary
            silver_dictionary = {'Show_Name':Show_Name,'Show_Type': Show_Type,'Theatre':Theatre,
                                'Theatre_Address':Theatre_Address, 'Prod_Type':Prod_Type,
                                'Run_Type':Run_Type,'Market':Market,'Intermission':Intermission, 
                                'Run_Time_String': Run_Time_String, 'Run_Time_Minutes':Run_Time_Minutes,
                                 'Date_Previews':Date_Previews, 'Date_Open':Date_Open, 'Date_Close': Date_Close}
            
            # append list of dictionaries
            dfs_silver_list.append(silver_dictionary)
            print('\t%d line has been scraped' %n)
        
        # What happens if no internet?
        except urllib.error.URLError as err:
            cprint('\t\t...! ...! ...! ...   N O   I N T E R N E T   for  line %d' %n,
                   'red', attrs=['bold'], file=sys.stderr)

        # What happens if manually interupted?
        except (KeyboardInterrupt,SystemExit):
            cprint('\n\t\tScript was interupted by a human for line %d' %n,
                   'red', 'on_blue',attrs=['bold'], file=sys.stderr)
        
        # What happens if machine rejects line?    
        except:
            cprint('\t\t\tLine %d could not be processed' %n,'red', attrs=['bold'], file=sys.stderr)
            lost_rows = lost_rows+1
            pass
        
        # Go onto next line.
        n = n+1
        # Update Progress Bar
    
    # How long did it take the function to process?
    end = time.time()
    total_minutes= ((end - start)/60)
    seconds_per_line = ((end - start)/n)
    print('\nFunction has run through lines 0 - %d!\n' %n,'Function took total of %0.2f minutes.\n' %total_minutes,
         '\tThat\'s %0.3f \tseconds per 1 lines.\n' %seconds_per_line,
         '\tThat\'s %0.3f \tminutes per 1,000 lines.'%(seconds_per_line*(1000/60)))
    cprint('\nA total of %d rows were lost.' %lost_rows, 'white','on_red')
    
    # Here's your data.
    silver_data = pd.DataFrame(dfs_silver_list)
    return silver_data
    

### Apply function in 3 stages
_Saves you a big headache if internet goes out for a portion or if any other interuptions are necessary*
*Of note, **function took YB a total of 5.5 hours for all three segments.**_

In [0]:
part0 = get_silver_data(show_links_nested[0][:5000])
today = datetime.date.today().strftime('%m-%d-%Y') 
part0.to_csv('broadway_silver_data_p0 {}.csv'.format(today))

In [0]:
part1 = get_silver_data(show_links_nested[0][5000:10000])
today = datetime.date.today().strftime('%m-%d-%Y') 
part1.to_csv('broadway_silver_data_p1 {}.csv'.format(today))

In [0]:
part2 = get_silver_data(show_links_nested[0][10000:])
today = datetime.date.today().strftime('%m-%d-%Y') 
part2.to_csv('broadway_silver_data_p2 {}.csv'.format(today))

In [0]:
broadway_silver_data = pd.concat([part0, part1, part2])

**Clean the data:**

In [0]:
cols = ['Show_Name','Run_Type','Prod_Type','Market','Theatre','Show_Type','Theatre_Address']
for col in broadway_silver_data:
    if col in cols:
        broadway_silver_data[col] = broadway_silver_data[col].astype('category')

In [0]:
date_cols = ['Date_Close','Date_Open', 'Date_Previews'] 
for col in broadway_silver_data:
    if col in date_cols:
        broadway_silver_data[col]  = pd.to_datetime(broadway_silver_data[col])

In [0]:
today = datetime.date.today().strftime('%m-%d-%Y') 
broadway_silver_data.to_csv('broadway_silver_data_MAIN {}.csv'.format(today))

## You have the data!

    broadway_silver_data

In [0]:
broadway_silver_data.info()

***
   >  
*** 
     
    